# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 4 2023 10:54AM,254265,16,TASA-284193,TASA USA Inc.,Released
1,Jan 4 2023 10:53AM,254264,20,8968296,"Exact-Rx, Inc.",Released
2,Jan 4 2023 10:53AM,254264,20,8968298,"Exact-Rx, Inc.",Released
3,Jan 4 2023 10:53AM,254264,20,8968305,"Exact-Rx, Inc.",Released
4,Jan 4 2023 10:53AM,254264,20,8968313,"Exact-Rx, Inc.",Released
5,Jan 4 2023 10:53AM,254264,20,8984631,"Exact-Rx, Inc.",Released
6,Jan 4 2023 10:53AM,254264,20,8984653,"Exact-Rx, Inc.",Released
7,Jan 4 2023 10:53AM,254264,20,8984655-C,"Exact-Rx, Inc.",Released
8,Jan 4 2023 10:53AM,254264,20,8984658,"Exact-Rx, Inc.",Released
9,Jan 4 2023 10:45AM,254263,12,8988995,Livs Products,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
34,254261,Released,7
35,254262,Released,1
36,254263,Released,1
37,254264,Released,8
38,254265,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254261,NaN,NaN,7.0
254262,NaN,NaN,1.0
254263,NaN,NaN,1.0
254264,NaN,NaN,8.0
254265,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254157,1.0,0.0,1.0
254171,58.0,43.0,11.0
254172,0.0,3.0,0.0
254187,0.0,0.0,3.0
254190,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254157,1,0,1
254171,58,43,11
254172,0,3,0
254187,0,0,3
254190,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254157,1,0,1
1,254171,58,43,11
2,254172,0,3,0
3,254187,0,0,3
4,254190,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254157,1,,1
1,254171,58,43,11
2,254172,,3,
3,254187,,,3
4,254190,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 4 2023 10:54AM,254265,16,TASA USA Inc.
1,Jan 4 2023 10:53AM,254264,20,"Exact-Rx, Inc."
9,Jan 4 2023 10:45AM,254263,12,Livs Products
10,Jan 4 2023 10:42AM,254262,16,TASA USA Inc.
11,Jan 4 2023 10:42AM,254261,10,"Methapharm, Inc."
18,Jan 4 2023 10:40AM,254260,10,Emerginnova
20,Jan 4 2023 10:29AM,254259,10,"Methapharm, Inc."
23,Jan 4 2023 10:22AM,254258,19,"Methapharm, Inc."
31,Jan 4 2023 10:22AM,254258,19,Methapharm-G
32,Jan 4 2023 10:18AM,254257,19,"CLINUVEL, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 4 2023 10:54AM,254265,16,TASA USA Inc.,,,1
1,Jan 4 2023 10:53AM,254264,20,"Exact-Rx, Inc.",,,8
2,Jan 4 2023 10:45AM,254263,12,Livs Products,,,1
3,Jan 4 2023 10:42AM,254262,16,TASA USA Inc.,,,1
4,Jan 4 2023 10:42AM,254261,10,"Methapharm, Inc.",,,7
5,Jan 4 2023 10:40AM,254260,10,Emerginnova,,,2
6,Jan 4 2023 10:29AM,254259,10,"Methapharm, Inc.",,,3
7,Jan 4 2023 10:22AM,254258,19,"Methapharm, Inc.",,,9
8,Jan 4 2023 10:22AM,254258,19,Methapharm-G,,,9
9,Jan 4 2023 10:18AM,254257,19,"CLINUVEL, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 10:54AM,254265,16,TASA USA Inc.,1,,
1,Jan 4 2023 10:53AM,254264,20,"Exact-Rx, Inc.",8,,
2,Jan 4 2023 10:45AM,254263,12,Livs Products,1,,
3,Jan 4 2023 10:42AM,254262,16,TASA USA Inc.,1,,
4,Jan 4 2023 10:42AM,254261,10,"Methapharm, Inc.",7,,
5,Jan 4 2023 10:40AM,254260,10,Emerginnova,2,,
6,Jan 4 2023 10:29AM,254259,10,"Methapharm, Inc.",3,,
7,Jan 4 2023 10:22AM,254258,19,"Methapharm, Inc.",9,,
8,Jan 4 2023 10:22AM,254258,19,Methapharm-G,9,,
9,Jan 4 2023 10:18AM,254257,19,"CLINUVEL, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 10:54AM,254265,16,TASA USA Inc.,1,,
1,Jan 4 2023 10:53AM,254264,20,"Exact-Rx, Inc.",8,,
2,Jan 4 2023 10:45AM,254263,12,Livs Products,1,,
3,Jan 4 2023 10:42AM,254262,16,TASA USA Inc.,1,,
4,Jan 4 2023 10:42AM,254261,10,"Methapharm, Inc.",7,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 10:54AM,254265,16,TASA USA Inc.,1.0,NaN,NaN
1,Jan 4 2023 10:53AM,254264,20,"Exact-Rx, Inc.",8.0,NaN,NaN
2,Jan 4 2023 10:45AM,254263,12,Livs Products,1.0,NaN,NaN
3,Jan 4 2023 10:42AM,254262,16,TASA USA Inc.,1.0,NaN,NaN
4,Jan 4 2023 10:42AM,254261,10,"Methapharm, Inc.",7.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 10:54AM,254265,16,TASA USA Inc.,1.0,0.0,0.0
1,Jan 4 2023 10:53AM,254264,20,"Exact-Rx, Inc.",8.0,0.0,0.0
2,Jan 4 2023 10:45AM,254263,12,Livs Products,1.0,0.0,0.0
3,Jan 4 2023 10:42AM,254262,16,TASA USA Inc.,1.0,0.0,0.0
4,Jan 4 2023 10:42AM,254261,10,"Methapharm, Inc.",7.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2796552,48.0,4.0,0.0
12,254263,1.0,0.0,0.0
15,1016753,113.0,52.0,59.0
16,762779,3.0,0.0,0.0
19,2796421,81.0,14.0,0.0
20,508501,8.0,1.0,0.0
21,254210,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2796552,48.0,4.0,0.0
1,12,254263,1.0,0.0,0.0
2,15,1016753,113.0,52.0,59.0
3,16,762779,3.0,0.0,0.0
4,19,2796421,81.0,14.0,0.0
5,20,508501,8.0,1.0,0.0
6,21,254210,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,48.0,4.0,0.0
1,12,1.0,0.0,0.0
2,15,113.0,52.0,59.0
3,16,3.0,0.0,0.0
4,19,81.0,14.0,0.0
5,20,8.0,1.0,0.0
6,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,48.0
1,12,Released,1.0
2,15,Released,113.0
3,16,Released,3.0
4,19,Released,81.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,0.0,0.0,59.0,0.0,0.0,0.0,0.0
Executing,4.0,0.0,52.0,0.0,14.0,1.0,0.0
Released,48.0,1.0,113.0,3.0,81.0,8.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,59.0,0.0,0.0,0.0,0.0
1,Executing,4.0,0.0,52.0,0.0,14.0,1.0,0.0
2,Released,48.0,1.0,113.0,3.0,81.0,8.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,59.0,0.0,0.0,0.0,0.0
1,Executing,4.0,0.0,52.0,0.0,14.0,1.0,0.0
2,Released,48.0,1.0,113.0,3.0,81.0,8.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()